In [293]:
#import statements
import pandas as pd 
import numpy as np 
import pickle
pd.set_option('display.max_columns', None)

Read in data   
Read in those properties that fall under "Multi Family" and "Single Family"

**Find the data codebook here**:   https://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/?view_287_page=1

**Columns to keep**:  'assessment_date', 'basements', 'building_code_description',
                 'category_code_description','census_tract', 'central_air','depth',
                 'exterior_condition', 'fireplaces', 'frontage', 'fuel', 'garage_spaces',
                 'geographic_ward', 'house_number', 
                 'interior_condition',
                 'market_value', 'market_value_date', 'number_of_bathrooms', 
                 'number_of_bedrooms', 'number_of_rooms','number_stories', 'quality_grade', 
                 'sale_date', 'sale_price', 'street_designation',
                 'topography', 'total_area','total_livable_area', 'type_heater', 
                 'unfinished',  'view_type', 'year_built',
                 'zip_code',  'lat', 'lng'

In [309]:
#place columns of interest into a list
cols_interest = ['assessment_date', 'basements', 
                 'category_code_description','census_tract', 'central_air','depth',
                 'exterior_condition', 'fireplaces', 'frontage', 'fuel', 'garage_spaces',
                 'geographic_ward', 'house_number', 
                 'interior_condition',
                 'market_value', 'market_value_date', 'number_of_bathrooms', 
                 'number_of_bedrooms', 'number_of_rooms','number_stories', 'quality_grade', 
                 'sale_date', 'sale_price', 'street_designation',
                 'topography', 'total_area','total_livable_area', 'type_heater', 
                 'unfinished',  'view_type', 'year_built',
                 'zip_code',  'lat', 'lng']

In [310]:
#read in data using columns of interest 
#query only those multifamily and single family homes
dat = (pd.read_csv("/home/jovyan/work/Philadelphia-Housing/processing/opa_properties_public.csv", 
                 usecols = cols_interest)
       .query('category_code_description == "Multi Family" | category_code_description == "Single Family"')
)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (67,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We re-code four columns of interest (basements, central_air, fuel, topography, type_heater, street_designation).    

In [311]:
dat['street_designation'].value_counts()

ST      371811
AVE      73019
RD       29498
LA        7600
DR        5777
PL        5651
SQ        2369
BLV       2212
TER       1595
CT        1135
WAY       1065
CIR        776
LN         569
PK         497
PKY        344
PLZ        168
BLVD       126
MEW         79
ALY         59
WLK         55
PIKE        29
HTS         22
PTH         12
PKWY         9
MEWS         7
WALK         5
ROW          4
PATH         3
ML           3
MALL         1
Name: street_designation, dtype: int64

In [312]:
dat['basements'] = dat['basements'].map({'A': 'full', 'B': 'full','C': 'full','D': 'full','I': 'full',
                                        'E': 'partial','F': 'partial','G': 'partial','H': 'partial','J': 'partial',
                                        '0': 'None'})
dat['central_air'] = dat['central_air'].map({'0': 'N', '1': 'Y','Y': 'Y', 'N': 'N'})

dat['fuel'] = dat['fuel'].map({'A': 'NG', 'B': 'Oil','C': 'Electric',
                              'E': 'other', 'G': 'other','H': 'other','I':'other'})
dat['topography'] = dat['topography'].map({'A': 'A', 'B': 'B','C': 'C',
                              'D': 'D', 'E': 'E','F': 'F'})

dat['type_heater'] = dat['type_heater'].map({'A': 'A', 'B': 'B','C': 'C',
                              'D': 'D', 'E': 'E','G': 'G','H': 'H'})

dat['street_designation'] = dat['street_designation'].astype("string")

dat['street_designation'] = np.where(dat['street_designation'].str.match("ST"), "ST",
                                  np.where(dat['street_designation'].str.match("AVE"), "AVE",
                                  np.where(dat['street_designation'].str.match("RD"), "RD",
                                  np.where(dat['street_designation'].str.match("LA"), "LA",
                                  np.where(dat['street_designation'].str.match("DR"), "DR",
                                  np.where(dat['street_designation'].str.match("PL"), "PL",
                                  np.where(dat['street_designation'].str.match("SQ"), "SQ","Other")))))))

Below, we output the percentage missing by column:  
Note that assessment_date, fuel, market_value_date, quality_grade, and unfinished are all over ~90% missing.   
For these reasons, we will drop these columns from the analysis, as they will not provide robust and useful information for the analysis and prediction process.  

In [313]:
dat.isnull().sum() * 100 / len(dat)

assessment_date               93.949851
basements                     36.815857
category_code_description      0.000000
census_tract                   0.006938
central_air                   44.558375
depth                          0.077899
exterior_condition             0.088206
fireplaces                     0.164718
frontage                       0.079088
fuel                          97.111992
garage_spaces                  0.199405
geographic_ward                0.006938
house_number                   0.000000
interior_condition             0.109812
market_value                   0.018236
market_value_date            100.000000
number_of_bathrooms            0.143707
number_of_bedrooms             0.083845
number_of_rooms                5.908622
number_stories                 0.083251
quality_grade                 89.448167
sale_date                      0.001586
sale_price                     0.002180
street_designation             0.000000
topography                     6.621011


In [314]:
dat = dat.drop(['assessment_date', 'fuel', 
         'market_value_date', 'quality_grade', 
         'unfinished', 'central_air', 'type_heater'], axis = 1)

Next, we will remove those datapoints/rows that do not have values for **sale price** or **sale date**.   
These are removed since these are our outcome variables.  
Both the sales prices and the date which the house was sold is necessary to perform our experiment.  

In [315]:
#drop the rows which 'sale_date' is NAN
dat = dat[dat['sale_date'].notna()]
#drop the rows which 'sale_price' is NAN
dat = dat[dat['sale_price'].notna()]

Find the percentage of homes that have a sales price of greater than \\$5 million 

In [316]:
(dat['sale_price'] > 5_000_000).mean() *100

0.2820675971131184

*Note*: 0.28% of the data has a sale_price of greater than \\$5 million.   
We remove these from the dataset since we are not interested in these homes.  

In [317]:
dat = dat[dat['sale_price'] < 5_000_000]

Find the percentage of homes that have a sales price of $1

In [318]:
(dat['sale_price'] == 1).mean() *100

26.218538846734162

In [326]:
(dat['sale_price']  < 1000).mean() *100

28.612975880693618

Note: 26% of the data has a sale_price of \\$1.   
Further, 28.6 of the data has a sales price of \\$1000.   
We remove these from the dataset since we believe these sales prices were not indicative of true sales.   
We are also not interested in those homes sold for \\$1.   

In [327]:
dat = dat[dat['sale_price'] > 1000]

In [328]:
dat['sale_price'].describe()

count    3.582260e+05
mean     1.693408e+05
std      2.913644e+05
min      1.001000e+03
25%      4.100000e+04
50%      9.500000e+04
75%      1.950000e+05
max      4.998000e+06
Name: sale_price, dtype: float64

Next, we deal with dates:    
First we coerce the dates    
Next we will break apart the date elements into variables   

In [329]:
#create a date variable
dat['sale_date'] = pd.to_datetime(dat['sale_date'])
#break apart the date to new variables:  
dat['sale_year'] = dat['sale_date'].dt.year
dat['sale_month'] = dat['sale_date'].dt.month
dat['sale_week'] = dat['sale_date'].dt.isocalendar().week
dat['sale_day'] = dat['sale_date'].dt.day
dat['sale_dow'] = dat['sale_date'].dt.dayofweek
#change the sale week column type to int since it is coerced originally to 
#UInt32
dat['sale_week'] = dat['sale_week'].astype(int)

Certain 'year_built' inputs are incorrect.    
We ensure to change these below:   

In [330]:
dat['year_built'] = dat['year_built'].replace('196Y', np.NaN)
#convert to numeric
dat["year_built"] = pd.to_numeric(dat["year_built"])
#replace year 0 with NaN
dat['year_built'] = dat['year_built'].replace(0, np.NaN)

In [331]:
dat['year_built'].describe(include = 'all')

count    357830.000000
mean       1938.207068
std          28.615644
min        1652.000000
25%        1920.000000
50%        1928.000000
75%        1952.000000
max        2022.000000
Name: year_built, dtype: float64

In [332]:
dat['year_built'].isnull().sum() * 100 / len(dat)

0.11054473991279248

We will now perform preliminary train/test processing of the data necessary for modeling    

### Training / Test Split

First, we will split the data into training and test sets:   
The training dataset contains all housing prices that were sold during the years of 2010 - 2019.    
The test dataset contains all housing prices that were sold during 2020 and 2021.  

In [333]:
#first subset the data only to 2010 - 2021
dat = dat[dat['sale_year'] > 2009]
#split to the training set 
train = dat[dat['sale_year'] < 2020]
#split to the test set 
test = dat[dat['sale_year'] > 2019]

#finally, drop the date column:  
train = train.drop(['sale_date'], axis=1)
test = test.drop(['sale_date'], axis=1)

In [334]:
train.shape

(150594, 31)

In [335]:
test.shape

(29614, 31)

In [336]:
train.shape[0] / (train.shape[0] + test.shape[0])

0.8356676729113025

The training data represents approximately 84% of the entire data.   
The test data represents 26%   

We pickle the training data that has not yet been processed for visualization purposes

In [337]:
pickle_out = open("housedat_train_vis.pickle","wb")
pickle.dump(train, pickle_out)
pickle_out.close()

#### Processing part 2
Next, we process the training data via one hot encoding

In [338]:
# convert categorical variables to dummy variables and add dummy variables to data frame
cat_vars = train[['basements','category_code_description','street_designation',
               'topography', 'view_type']]
cat_dummies = pd.get_dummies(cat_vars, drop_first=True)
train = train.drop(['basements','category_code_description','street_designation',
               'topography', 'view_type'], axis=1)
train = pd.concat([train, cat_dummies], axis=1)

In [339]:
train = train.fillna(train.mean())

In [340]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train.drop(['sale_price'], axis=1),
           train['sale_price'])
reg.score(train.drop(['sale_price'], axis=1),
           train['sale_price'])

0.29815987514127607

In [341]:
import statsmodels.api as sm
model = sm.OLS(train['sale_price'], 
               train.drop(['sale_price'], axis=1)
           )
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             sale_price   R-squared (uncentered):                   0.511
Model:                            OLS   Adj. R-squared (uncentered):              0.511
Method:                 Least Squares   F-statistic:                              3347.
Date:                Thu, 07 Apr 2022   Prob (F-statistic):                        0.00
Time:                        02:25:03   Log-Likelihood:                     -2.1030e+06
No. Observations:              150594   AIC:                                  4.206e+06
Df Residuals:                  150547   BIC:                                  4.206e+06
Df Model:                          47                                                  
Covariance Type:            nonrobust                                                  
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
census_tract                             -542.0122     12.460    -43.501      0.000    -566.433    -517.591
depth                                       2.2588      1.652      1.367      0.171      -0.979       5.496
exterior_condition                       -1.33e+04   3855.868     -3.449      0.001   -2.09e+04   -5739.806
fireplaces                               1.307e+05   2687.158     48.628      0.000    1.25e+05    1.36e+05
frontage                                    0.3336      0.063      5.330      0.000       0.211       0.456
garage_spaces                            2.035e+04    960.676     21.186      0.000    1.85e+04    2.22e+04
geographic_ward                         -1241.4575     49.592    -25.034      0.000   -1338.656   -1144.259
house_number                               -3.3164      0.256    -12.973      0.000      -3.818      -2.815
interior_condition                      -4.874e+04   3831.887    -12.720      0.000   -5.63e+04   -4.12e+04
market_value                                0.1034      0.001     78.353      0.000       0.101       0.106
number_of_bathrooms                      3.125e+04   1270.482     24.599      0.000    2.88e+04    3.37e+04
number_of_bedrooms                      -7195.0285    821.403     -8.759      0.000   -8804.962   -5585.095
number_of_rooms                         -8326.5364    392.414    -21.219      0.000   -9095.660   -7557.413
number_stories                           1.994e+04    388.346     51.335      0.000    1.92e+04    2.07e+04
total_area                                  2.0949      0.108     19.339      0.000       1.883       2.307
total_livable_area                         -0.5497      0.111     -4.950      0.000      -0.767      -0.332
year_built                                359.5319     28.036     12.824      0.000     304.582     414.482
zip_code                                -1076.7367     49.340    -21.823      0.000   -1173.442    -980.031
lat                                      2.115e+05   1.39e+04     15.210      0.000    1.84e+05    2.39e+05
lng                                      3.678e+05   2.19e+04     16.791      0.000    3.25e+05    4.11e+05
sale_year                                1.081e+04    259.208     41.694      0.000    1.03e+04    1.13e+04
sale_month                               -819.1448   1016.591     -0.806      0.420   -2811.642    1173.352
sale_week                                 785.0598    233.175      3.367      0.001     328.042    1242.078
sale_day                                  960.3489     85.510     11.231      0.000     792.750    1127.947
sale_dow                                -1475.6961    476.804     -3

In [289]:
import sklearn
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# data_dmatrix = xgb.DMatrix(train.drop(['sale_price'], axis=1),
#             train['sale_price'])

# xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                 max_depth = 5, alpha = 10, n_estimators = 10)
# regr = RandomForestRegressor()
# regr.fit(train.drop(['sale_price'], axis=1),
#            train['sale_price'])

# rfpreds = regr.predict(train.drop(['sale_price'], axis=1))
# mean_squared_error(train['sale_price'], rfpreds)

# xg_reg.fit(train.drop(['sale_price'], axis=1),
#             train['sale_price'])

# preds = xg_reg.predict(train.drop(['sale_price'], axis=1))

In [342]:
cat_vars_test = test[['basements','category_code_description','street_designation',
               'topography', 'view_type']]
cat_dummies_test = pd.get_dummies(cat_vars_test, drop_first=True)

test = test.drop(['basements','category_code_description','street_designation',
               'topography', 'view_type'], axis=1)
test = pd.concat([test, cat_dummies_test], axis=1)

test = test.fillna(test.mean())

In [348]:
rpreds = reg.predict(test.drop(['sale_price'], axis=1))
mean_squared_error(test['sale_price'], rpreds)

82704113290.46231

We then pickle the dataset for later use and easy loading:    

In [ ]:
pickle_out = open("housedat.pickle","wb")
pickle.dump(dat, pickle_out)
pickle_out.close()

possible predictions datasets:   
https://www.kaggle.com/datasets/harlfoxem/housesalesprediction     
https://github.com/michellesklee/predicting_home_values    